In [1]:
import torch
import torchvision
import numpy as np
from sklearn.model_selection import train_test_split
from torchvision import transforms
from collections import Counter
import matplotlib.pyplot as plt
import torchvision.models as models
from tqdm import tqdm
import torch.nn as nn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import random

In [2]:
SEED = 1
# Use GPU if available, otherwise CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(device)

mps


In [3]:
imagenette_transforms = transforms.Compose(
    [
        transforms.Resize((224,224)),
        transforms.ToTensor()
    ]
)

In [4]:
data_train = torchvision.datasets.Imagenette(root='./datasets/imagenette', split='train', download=False, transform=imagenette_transforms)
data_val = torchvision.datasets.Imagenette(root='./datasets/imagenette', split='val', download=False, transform=imagenette_transforms)

In [5]:
class ImagenetteDataset(torch.utils.data.Dataset):

    def __init__(self, data, transform=None):
        self.transforms = transform
        self.data = data

    def __getitem__(self,index):
        image,label = self.data[index]
        if self.transforms != None:
            image = self.transforms(image)

        label = 1 if label==3 else 0

        return image,label

    def __len__(self):
        return len(self.data)

In [6]:
data_train_chainsaw = ImagenetteDataset(data_train)
data_val_chainsaw = ImagenetteDataset(data_val)

In [7]:
train_loader = torch.utils.data.DataLoader(data_train_chainsaw, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(data_val_chainsaw, batch_size=64, shuffle=False)

In [8]:
for b,c in train_loader:
    for d,e in zip(b,c):
        transform = transforms.ToPILImage()
        img = transform(d)
        # img.show()
        print(e)
    break

tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(1)
tensor(1)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(1)
tensor(0)
tensor(0)
tensor(1)
tensor(1)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(1)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(1)
tensor(0)
tensor(0)
tensor(1)
tensor(0)
tensor(0)
tensor(1)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(1)
tensor(0)
tensor(1)
tensor(0)
tensor(0)
tensor(0)


In [9]:
class Resnet18(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.backbone = models.resnet18(weights='DEFAULT')
        self.backbone.fc = nn.Identity()
        self.backbone.to(device)
        self.head = nn.Linear(512, num_classes)
        self.head.to(device)

    def forward(self, x):
        x = self.backbone(x)
        x = self.head(x)
        return x

In [10]:
model = Resnet18(num_classes=2)
lr = 0.008
#criterion = nn.BCELoss()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/benlin/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:01<00:00, 39.5MB/s]


In [11]:
def train(model,data_loader,criterion,optimizer,attack=False):

    running_loss = 0.
    running_steps = 0
    n_correct = 0
    n_total = 0

    with tqdm(data_loader, desc ="   train") as train_tqdm:
        for batch_idx, (inputs, targets) in enumerate(train_tqdm):
            model.train() # add a call before calling train function
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            running_steps += 1

            _, pred = torch.max(outputs, 1)
            n_correct += (pred==targets).sum().item()
            n_total += len(targets)
    
    average_train_loss = running_loss / running_steps
    accuracy = n_correct / n_total * 100.

    return average_train_loss, accuracy

In [12]:
@torch.no_grad()
def test(model,data_loader,criterion,optimizer,attack=False):
    model.eval()

    running_loss = 0.
    running_steps = 0
    n_correct = 0
    n_total = 0

    with tqdm(data_loader, desc ="   test") as test_tqdm:
        for batch_idx, (inputs, targets) in enumerate(test_tqdm):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)

            loss = criterion(outputs, targets)
            running_loss += loss.item()
            running_steps += 1

            _, pred = torch.max(outputs, 1)
            n_correct += (pred==targets).sum().item()
            n_total += len(targets)
    
    average_test_loss = running_loss / running_steps
    accuracy = n_correct / n_total * 100.0

    return average_test_loss, accuracy

In [13]:
for i in range(2):
    train_avg_loss, train_accuracy = train(model, train_loader, criterion, optimizer)
    if (i+1)%10 == 0:
        val_avg_loss, val_accuracy = test(model, val_loader, criterion, optimizer)
        print('Epoch: {:3d}, Train Average Loss: {:.2f}, Train Accuracy: {:.1f}%, Validation Average Loss: {:.2f}, Validation Accuracy: {:.1f}%'\
              .format(i+1, train_avg_loss, train_accuracy, val_avg_loss, val_accuracy))

   train: 100%|██████████| 148/148 [01:38<00:00,  1.51it/s]


In [14]:
val_avg_loss, val_accuracy = test(model, val_loader, criterion, optimizer)

   test: 100%|██████████| 62/62 [00:23<00:00,  2.58it/s]


In [15]:
print('Validation Average Loss: {:.2f}, Validation Accuracy: {:.1f}%'\
              .format(val_avg_loss, val_accuracy))

Validation Average Loss: 0.30, Validation Accuracy: 90.2%


In [30]:
shape = data_val[50][0].shape
out = model.backbone(torch.reshape(data_val[50][0].to(device), (1, *shape)))

In [37]:
params = list(model.backbone.parameters())
weight = np.squeeze(params[-2].data.cpu().numpy())
conv_shape = out.shape
print(out.shape)
print()
cam = weight.dot(out.cpu().detach().numpy().reshape((conv_shape[0]*conv_shape[1])))
cam = cam.reshape(shape[1], shape[2])

import cv2

heatmap = cv2.applyColorMap(cv2.resize(cam, (shape[1], shape[2])), cv2.COLORMAP_JET)
result = heatmap*0.3 + data_val[50][0]*0.5

plt.imshow(result)

torch.Size([1, 512])



ValueError: cannot reshape array of size 1 into shape (224,224)